In [1]:
%%capture
# ! pip install compyle
# ! pip install pyopencl
# ! pip install pycuda

In [2]:
import numpy as np
import numba as nb
from numba import uint32,uint64,njit
from numba.experimental import jitclass
# from compyle.api import annotate,wrap,get_config
from time import perf_counter
import matplotlib.pyplot as plt

maxInt = np.uint32(4294967295)

## Tesing Python's inbuilt 'int' type

In [3]:
tlist = []
p = 0
q = 0
for i in range(1024):
    p = p+(2**i)
    q = q+(2**i)
    tstart = perf_counter()
    c = p+q
    texec = perf_counter()-tstart
    tlist.append(texec)

In [4]:
%timeit c=p+q

203 ns ± 104 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Custom implemetation of arithmetic on large size integers, with numpy arrays

In [5]:
def add(x, y, res):
    carry = 0
    for i in range(32):
        res[i] = x[i]+carry
        carry = (res[i]<x[i])
        res[i] = y[i]+res[i]
        carry = (res[i]<y[i] or carry)

In [6]:
x1 = np.zeros((32,),dtype='uint32')
y1 = np.zeros((32,),dtype='uint32')
z1 = np.zeros((32,),dtype='uint32')
x1[:-1] = maxInt
x1[31] = maxInt/2
y1[:-1] = maxInt
y1[31] = maxInt/2

# backend = 'cython'
# get_config().use_openmp = True
# x1, y1, z1 = wrap(x1, y1, z1, backend=backend)

In [7]:
add(x1,y1,z1)
print(np.flip(z1))
%timeit add(x1,y1,z1)

[4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967294]


<ipython-input-5-80b6a2b14194>:6: RuntimeWarning: overflow encountered in uint_scalars
  res[i] = y[i]+res[i]


128 µs ± 37.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Optimizations using numba with jitclass

In [8]:
@jitclass([
    ('arrSize', uint32),               # a simple scalar field
    ('array', uint32[:]),          # an array field
])
class BigInt:
    def __init__(self):
        self.arrSize = 0
        self.array = np.zeros(32, dtype=np.uint32)


    def storeArr(self, arr):
        self.arrSize = arr.shape[0]
        for i in range(self.arrSize):
            self.array[i] = arr[i]


    def add(x, y, res):
        carry = 0
        for i in range(min(x.arrSize,y.arrSize)):
            res.array[i] = y.array[i]+x.array[i]+carry
            if res.array[i]<x.array[i] or res.array[i]<y.array[i]:  #overflow
                carry = 1
            else:
                carry = 0

        if x.arrSize>y.arrSize:
            for i in range(y.arrSize,x.arrSize):
                res.array[i] = x.array[i] + carry
                carry = 1 if res.array[i]==0 else 0
            res.array[x.arrSize-1] = carry
            res.arrSize = x.arrSize + carry
        else:
            for i in range(x.arrSize,y.arrSize):
                res.array[i] = y.array[i] + carry
                carry = 1 if res.array[i]==0 else 0
            res.array[y.arrSize-1] = carry
            res.arrSize = y.arrSize + carry

        for i in range(max(x.arrSize,y.arrSize)+carry,32):
            res.array[i]=0


    def subtract(x, y, res):
        borrow = 0
        sz = x.arrSize if x.arrSize>=y.arrSize else 32
        for i in range(sz):
            if (borrow==0 or x.array[i]>0) and x.array[i]-borrow>=y.array[i]:
                res.array[i] = (x.array[i]-borrow)-y.array[i]
                borrow = 0
            else:
                res.array[i] = x.array[i]+(maxInt-y.array[i])+(1-borrow)
                borrow = 1

        for i in range(sz,32):
            res.array[i] = maxInt*borrow

        return borrow

    def multiply(x, y, res):
        xl = BigInt()

In [9]:
a = BigInt()
b = BigInt()
c = BigInt()
a1 = np.zeros((32,),dtype='uint32')
b1 = np.zeros((32,),dtype='uint32')
a1[0:-1] = maxInt
a1[31] = maxInt/2
b1[0:-1] = maxInt
b1[31] = maxInt/2
a.storeArr(a1)
b.storeArr(b1)

In [10]:
BigInt.add(a,b,c)
%timeit BigInt.add(a,b,c)
print(np.flip(c.array))

<ipython-input-8-20e517936b32>:20: RuntimeWarning: overflow encountered in uint_scalars
  res.array[i] = y.array[i]+x.array[i]+carry
<ipython-input-8-20e517936b32>:20: RuntimeWarning: overflow encountered in uint_scalars
  res.array[i] = y.array[i]+x.array[i]+carry


324 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
[         0 4294967294 4294967295 4294967294 4294967295 4294967294
 4294967295 4294967294 4294967295 4294967294 4294967295 4294967294
 4294967295 4294967294 4294967295 4294967294 4294967295 4294967294
 4294967295 4294967294 4294967295 4294967294 4294967295 4294967294
 4294967295 4294967294 4294967295 4294967294 4294967295 4294967294
 4294967295 4294967294]


In [11]:
a1[:] = 0
b1[:] = maxInt
a.storeArr(a1)
b.storeArr(b1)

In [12]:
borrow = BigInt.subtract(b,a,c)
%timeit BigInt.subtract(b,a,c)
print(borrow, np.flip(c.array))

330 µs ± 7.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0 [4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295]


## Optimations using numba with njit over numpy arrays (with size specified)

In [13]:
@njit
def add(x, y, res):
    carry = 0
    for i in range(min(x[32],y[32])):
        res[i] = y[i]+x[i]+carry
        if res[i]<x[i] or res[i]<y[i]:  #overflow
            carry = 1
        else:
            carry = 0

    if x[32]>y[32]:
        for i in range(y[32],x[32]):
            res[i] = x[i] + carry
            carry = 1 if res[i]==0 else 0
        res[x[32]] = carry
        res[32] = x[32] + carry
    else:
        for i in range(x[32],y[32]):
            res[i] = y[i] + carry
            carry = 1 if res[i]==0 else 0
        res[y[32]] = carry
        res[32] = y[32] + carry

    for i in range(max(x[32],y[32])+carry,32):
        res[i]=0

In [14]:
x1 = np.zeros((33,),dtype='uint32')
y1 = np.zeros((33,),dtype='uint32')
z1 = np.zeros((33,),dtype='uint32')
x1[:-2] = maxInt
x1[31] = maxInt/2
x1[32] = 32
y1[:-2] = maxInt
y1[31] = maxInt/2
y1[32] = 32
print(np.flip(x1))

[        32 2147483647 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295]


In [15]:
add(x1,y1,z1)
print(np.flip(z1))
%timeit add(x1,y1,z1)

[        32 4294967295 4294967294 4294967295 4294967294 4294967295
 4294967294 4294967295 4294967294 4294967295 4294967294 4294967295
 4294967294 4294967295 4294967294 4294967295 4294967294 4294967295
 4294967294 4294967295 4294967294 4294967295 4294967294 4294967295
 4294967294 4294967295 4294967294 4294967295 4294967294 4294967295
 4294967294 4294967295 4294967294]
837 ns ± 266 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Optimations using numba with njit over numpy arrays (64-bit)

In [16]:
@njit
def add64(x, y, res):
    carry = 0
    for i in range(16):
        res[i] = x[i]+carry
        carry = (res[i]<x[i])
        res[i] = y[i]+res[i]
        carry = (res[i]<y[i] or carry)

In [17]:
x2 = np.zeros((16,),dtype='uint64')
y2 = np.zeros((16,),dtype='uint64')
z2 = np.zeros((16,),dtype='uint64')
x2[:-1] = maxInt*2+1
x2[15] = maxInt
y2[:-1] = maxInt*2+1
y2[15] = maxInt
print(np.flip(x2))

[4294967295 8589934591 8589934591 8589934591 8589934591 8589934591
 8589934591 8589934591 8589934591 8589934591 8589934591 8589934591
 8589934591 8589934591 8589934591 8589934591]


In [18]:
add64(x2,y2,z2)
print(np.flip(z2))
%timeit add64(x2,y2,z2)

[ 8589934590 17179869182 17179869182 17179869182 17179869182 17179869182
 17179869182 17179869182 17179869182 17179869182 17179869182 17179869182
 17179869182 17179869182 17179869182 17179869182]
616 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Optimations using numba with njit over numpy arrays (simplified conditionals)

In [19]:
@njit
def add(x, y, res):
    carry = 0
    for i in range(32):
        res[i] = x[i]+carry
        carry = (res[i]<x[i])
        res[i] = y[i]+res[i]
        carry = (res[i]<y[i] or carry)

In [20]:
x1 = np.zeros((32,),dtype='uint32')
y1 = np.zeros((32,),dtype='uint32')
z1 = np.zeros((32,),dtype='uint32')
x1[:-1] = maxInt
x1[31] = maxInt/2
y1[:-1] = maxInt
y1[31] = maxInt/2
print(np.flip(x1))

[2147483647 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295]


In [21]:
add(x1,y1,z1)
print(np.flip(z1))
%timeit add(x1,y1,z1)

[4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967294]
709 ns ± 151 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
@njit
def subtract(x, y, res):
    borrow = 0
    for i in range(32):
        if (borrow==0 or x[i]>0) and x[i]-borrow>=y[i]:
            res[i] = (x[i]-borrow)-y[i]
            borrow = 0
        else:
            res[i] = x[i]+(maxInt-y[i])+(1-borrow)
            borrow = 1
    return borrow

In [23]:
x1 = np.zeros((32,),dtype='uint32')
y1 = np.zeros((32,),dtype='uint32')
z1 = np.zeros((32,),dtype='uint32')
y1[:] = maxInt

In [24]:
borrow = subtract(x1,y1,z1)
print(borrow, np.flip(z1))
%timeit subtract(x1,y1,z1)

1 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
662 ns ± 102 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [25]:
# partial products
@njit
def pp(i, x, y, p):
    temp = np.zeros((1,),'uint64')
    c = np.zeros((1,),'uint32')
    for j in range(32-i):
        temp[0] = x[i]*y[j]
        p[i+j] = c[0]+temp[0]
        c[0] = temp[0]>>32

In [26]:
# multiply (uses pp and add)
@njit
def multiply(x, y, res):
    temp = np.zeros((32,),'uint32')
    for i in range(31):
        p = np.zeros((32,),'uint32')
        pp(i,x,y,p)
        add(temp,p,res)
        temp = res.copy()

In [27]:
x1 = np.zeros((32,),dtype='uint32')
y1 = np.zeros((32,),dtype='uint32')
z1 = np.zeros((32,),dtype='uint32')
x1[0:15] = maxInt
y1[0:15] = maxInt
print(np.flip(x1))

[         0          0          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295]


In [28]:
temp = np.zeros((32,),dtype='uint32')
pp(14,x1,y1,temp)
print(np.flip(temp))

[         0          0 4294967294 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295          1
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0]


In [29]:
multiply(x1,y1,z1)
print(np.flip(z1))
%timeit multiply(x1,y1,z1)

[         0          0 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967295 4294967295
 4294967295 4294967295 4294967295 4294967295 4294967294          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          1]
17.7 µs ± 3.16 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
# python's native integer multiplication
%timeit c=p*q

1.49 µs ± 83.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Summay of results
 - Native python addition of 1024 bit integers takes about 200 ns
 - Trying to replicate the behaviour with numpy arrays gives us about 130 ns (500x slower than native python). We try to improve on this so that further algorithms can use the optimized implementation, compatible with numba/compyle
 - Using jitclass, there is a reduction in performance, indicating that it is not as well optimized and has additional overhead
 - We then try implementations with simple numpy arrays optimized using numba with njit
 - The final optimizations give the approximate average times (with worst case input combinations) for 1024 bit arithmetic with numpy arrays of uint32 elements as follows

| Operation | Native python integer type | Basic numpy (unoptimized) | Numba/compyle compatible (optimized) |
| --- | --- | --- | --- |
| Addition/subtraction | 203 ns  | 128 us | 709 ns  |
| Multiplication       | 1.49 us | -      |17.7 us | 
